# 任务2

## 问题分析

### 该任务要完成什么？

对系统进行仿真，求解两个仿真问题的数值解：

1. 求解节点个数 n，使得系统的可靠性最大
2. 求解节点个数 n，使得系统的平均工作寿命最大

具体而言，需要求解如下优化问题：

<center>
<figure>
<img src="q1.png" style="zoom:50%">
<img src="q2.png" style="zoom:50%">
</figure>
</center>

### 物理模型分析

- n个独立节点，节点内部物理同构
- 节点之间必须保持严格的同步时钟信号
- 每一时刻应只有1个主节点，工作于主模式中，其余为从节点，工作在从模式中

> 那么，这n个节点用什么方式来保存？最直觉的方式是使用`struct`，其次是使用多个向量来控制。
> 
> 目前暂且不明确用哪一种方式，`struct`和`repeat()`语法如下：
> 
> ```shell
> julia> struct Unit
>            x::Float64
>            y::Float64
>        end
> 
> julia> unit = repeat([Unit(1,9)],1,3)
> 1×3 Matrix{Unit}:
>  Unit(1.0, 9.0)  Unit(1.0, 9.0)  Unit(1.0, 9.0)
> ```

- 对于任意一个节点，分为主从两类，内部都存在`switchA`和`switchB`，而应对不同的模式，对于正常工作的判别是不同的，这里可以专门写个函数来判断（为简化主代码模块）
- 节点模式的重选原则：
  
  主模式$\rightarrow$从模式条件：
  
  - 内部的时钟检测线路发现，总线时钟信号异常（失效或不同于本地时钟源），退出主模式转为从模式。

  从模式$\rightarrow$主模式条件：
  
  - 在内部的时钟检测线路，检测到总线时钟信号失效（即总线时钟失效或主模式节点退出后），控制电路进入戒备状态
  - 戒备状态开始计时，计时长度取决于预设算法设定，与随机值相关。
  - 在当前节点的计时期间，如果时钟检测信号检测到有效的总线时钟，退出戒备，继续从模式
  - 在当前节点的计时结束后，如果时钟检测信号仍然未检测到有效总线时钟，则转为主动模式

  > 所以，可以比较明显的看到，不同节点所带有的不同随机值将会决定系统是否能继续处于正常工作状态。
  > 
  > 除此以外，这里计时可能最好不要使用真实计时，这样对于时间消耗太大了，可能就直接比较不同节点的等待时长，选择等待时长最短的。
  > 
  > 从这里可以看出，节点最好不用`struct`来构建，因为需要对节点的同一属性进行查询，直接`findmin()`可比循环查找要简洁多了。

  > 对于节点的切换，可以用以下公式表达：
  > $$
  > \{G_{role}^{(m)}\}=F_{G}(\{G_{role}^{(m-1)}\},\{G_{N}^{(m)}\},[random])
  > $$

### 实现分析

#### 节点

对于每一个节点，应当至少由以下元件部分组成：

- 切换器A `switchA`

  不可靠元件，存在失效风险。
  - 工作状态 `[0,1,2,3]`
    - 正常A0
    - 故障A1：掷刀无法脱离触点1
    - 故障A2：掷刀无法脱离触点2
    - 故障A3：掷刀无法接合任意触点
  - 掷刀位置 `[1,2,3]`
    - 触点1
    - 触点2
    - 未接合

- 切换器B `switchB`

  不可靠元件，存在失效风险。
  - 工作状态 `[0,1,2]`
    - 正常B0
    - 故障B1：掷刀无法脱离触点
    - 故障B2：掷刀无法接合触点
  - 掷刀位置 `[1,2]`
    - 接合
    - 未接合

- 节点性能状态 `[0,1,2,3,4,5]`
  
  节点性能状态完全由切换器状态决定，参考指导书表格
  - gN0: (gPF perfectly functioning)
  - gN1: (gSO slave only)
  - gN2: (gDM disable/master)
  - gN3: (gMO master only) 只能作为主节点，否则就会阻塞总线
  - gN4: (gDN disable node)
  - gN5: (gFB failed bus) 节点总是阻塞总线

![](node-state.png)

#### 系统

系统的工作状态完全由各节点状态的组合决定，而对于系统的状态类型可以通过不同类型状态节点来定义，需要统计各类节点的数目

> 这个相当于是把所有的实验要求都写出来了，只需要无脑写代码实现就好
> 
> 甚至不需要分析什么情况下处于工作状态等等的，关注点在于根据提示实现这个系统

这里我就不抄书了，没有分析的必要，写代码的时候照着书抄就可以了

### 参数设置

切换器元件使用寿命服从负指数分布，仿真粒度默认1h
- `switchA`
  $$
  f_{T_{A}}(\tau)=\lambda_{A}e^{-\lambda_{A}\cdot \tau}, ~~~~1/\lambda_{A}=5.90\times 10^4 hour
  $$
  正常工作的概率为：
  $$
  p_{A0}(t)=e^{-\lambda_{A}\cdot t}
  $$
  发生故障的概率为：
  $$
  p_{EA}(t)=1-e^{-\lambda_{A}\cdot t}
  $$
    在故障发生的条件下：
  $$
  \begin{aligned}
    P_{EA1}&=0.20 \\
    P_{EA2}&=0.15 \\
    P_{EA3}&=0.65 \\
  \end{aligned}
  $$

- `switchB`
  $$
  f_{T_{B}}(\tau)=\lambda_{B}e^{-\lambda_{B}\cdot \tau}, ~~~~1/\lambda_{B}=2.20\times 10^5 hour
  $$
  正常工作的概率为：
  $$
  p_{B0}(t)=e^{-\lambda_{B}\cdot t}
  $$
  发生故障的概率为：
  $$
  p_{EB}(t)=1-e^{-\lambda_{B}\cdot t}
  $$
  
  在故障发生的条件下：
  $$
  \begin{aligned}
    P_{EB1}&=0.45 \\
    P_{EB2}&=0.55 \\
  \end{aligned}
  $$


## 代码实现

### 一些参数的代码形式

首先是 NUM_SYSTEM 套同型系统的模拟，如果可以使用并行计算，可能有其他写法，这里留一个 TODO。

> Todo: 改用并行计算实现

否则，就只是连续计算 NUM_SYS 轮，在总代码里面就是套一个 S 的循环

```julia
NUM_SYSTEM=10_0000
LIFE_LIMIT = 20_0000
life = zeros(Integer,NUM_SYSTEM) # 我偏好于用列向量
for index=1:S
    # test certain time step
    ...
    while !state_system || cnt <= LIFE_LIMIT
        ...
    end
end

mean_life = mean(life)
```
